In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from tqdm import tqdm
from IPython.display import SVG

# RDKit for cheminformatics
from rdkit import Chem
from rdkit.Chem import AllChem, Draw, Descriptors, Lipinski, PandasTools
from rdkit.Chem.FilterCatalog import FilterCatalog, FilterCatalogParams
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import DataStructs
from rdkit.Avalon.pyAvalonTools import GetAvalonFP

# ChEMBL web resource client
from chembl_webresource_client.new_client import new_client 

# Machine learning libraries
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import (LinearRegression, Ridge, Lasso, ElasticNet)
from sklearn.ensemble import (RandomForestRegressor, GradientBoostingRegressor)
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import (r2_score, mean_absolute_error, mean_squared_error)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

# SHAP for model interpretation
import shap

# Padelpy for molecular descriptor calculations
from padelpy import padeldescriptor, from_smiles

# PubChem for chemical information retrieval
import pubchempy as pcp

# Additional libraries for data handling and visualization
import requests
import json
import math
import time  # To check time
from sklearn.model_selection import LeaveOneOut, cross_val_score


# Hide warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# Set display options for Pandas DataFrames
pd.set_option('display.max_columns', 1800)
pd.set_option('display.max_colwidth', None)  # None means no limit


In [2]:
#reloading my data under a new name which will be df2
df= pd.read_csv('ml_data.csv')
df = df.drop(df.columns[0], axis =1)
df.head()


,molecule_chembl_id,canonical_smiles,standard_value,bioactivity_class,pIC50,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nN,nO,nS,nF,nCl,nBr,nX,ATS0m,ATS0s,AATS0m,AATS1m,AATS0e,AATS6e,AATS8e,AATS0p,AATS0i,AATS1i,AATS3i,AATS4i,AATS6i,AATS8s,ATSC0c,ATSC1c,ATSC2c,ATSC3c,ATSC4c,ATSC5c,ATSC6c,ATSC7c,ATSC8c,ATSC1m,ATSC2m,ATSC3m,ATSC4m,ATSC5m,ATSC6m,ATSC7m,ATSC8m,ATSC1v,ATSC2v,ATSC3v,ATSC4v,ATSC5v,ATSC6v,ATSC7v,ATSC8v,ATSC1e,ATSC2e,ATSC3e,ATSC4e,ATSC5e,ATSC6e,ATSC7e,ATSC8e,ATSC1p,ATSC5p,ATSC7p,ATSC8p,ATSC1i,ATSC2i,ATSC3i,ATSC4i,ATSC5i,ATSC6i,ATSC7i,ATSC8i,ATSC1s,ATSC2s,ATSC3s,ATSC5s,ATSC6s,ATSC8s,AATSC0c,AATSC1c,AATSC2c,AATSC3c,AATSC4c,AATSC6c,AATSC7c,AATSC4m,AATSC6m,AATSC7m,AATSC8m,AATSC3v,AATSC4v,AATSC7v,AATSC4e,AATSC8e,AATSC0p,AATSC3p,AATSC4p,AATSC7p,AATSC0i,AATSC3i,AATSC4i,AATSC7i,AATSC8i,AATSC1s,AATSC3s,AATSC6s,AATSC8s,MATS1c,MATS3m,MATS1s,MATS2s,GATS2c,GATS3c,GATS4c,GATS5c,GATS6c,GATS7c,GATS8c,GATS2m,GATS3m,GATS4m,GATS5m,GATS6m,GATS7m,GATS8m,GATS2v,GATS3v,GATS7v,GATS8v,GATS1e,GATS2e,GATS3e,GATS4e,GATS5e,GATS6e,GATS7e,GATS2p,GATS3p,GATS5p,GATS6p,GATS1i,GATS2i,GATS3i,GATS5i,GATS6i,GATS8i,GATS1s,GATS2s,GATS3s,GATS4s,GATS5s,GATS6s,GATS7s,GATS8s,SM1_DzZ,VE1_DzZ,VE3_DzZ,VR1_DzZ,VE1_Dzv,VE2_Dzv,VE3_Dzv,VR1_Dzv,VE1_Dzp,VE3_Dzp,VR1_Dzp,VR1_Dzi,VE1_Dzs,VR1_Dzs,nBase,BCUTw-1l,BCUTc-1l,BCUTc-1h,BCUTp-1l,BCUTp-1h,nBondsD,nBondsT,SpMax2_Bhm,SpMax3_Bhm,SpMin1_Bhm,SpMin2_Bhm,SpMin3_Bhm,SpMin6_Bhm,SpMax1_Bhv,SpMin1_Bhv,SpMin8_Bhe,SpMax1_Bhs,SpMax2_Bhs,SpMax3_Bhs,SpMax4_Bhs,C1SP2,C2SP2,C3SP2,C1SP3,C2SP3,C3SP3,SCH-3,SCH-5,SCH-6,VCH-5,VCH-6,SC-4,SC-5,VC-3,VC-5,ASP-0,AVP-0,VE1_Dt,VE2_Dt,VE3_Dt,VR1_Dt,VR3_Dt,nHBd,nHBa,nHBint2,nHBint3,nHBint4,nHBint5,nHBint6,nHBint7,nHBint8,nHBint9,nHBint10,nHsOH,nHdNH,nHssNH,nHaaNH,nHdsCH,nHCsats,nHCsatu,nHAvin,nsCH3,ndssC,naasC,naaaC,ndsN,naaN,nsssN,naasN,ndO,nssO,nsOm,SHBint3,SHBint4,SHBint6,SHBint8,SHBint10,SdssC,SaasC,SaaaC,SssssC,minHBd,minHBa,minwHBa,minHBint2,minHBint3,minHBint4,minHBint5,minHBint6,minHBint7,minHBint8,minHBint10,minHsOH,minHdsCH,minHCsats,minsCH3,minaaCH,minaasC,minaaN,minsssN,minsF,maxHBd,maxHBa,maxHBint4,maxHaaCH,maxHother,maxdssC,maxaaN,hmax,hmin,gmin,ETA_dAlpha_A,ETA_dEpsilon_B,ETA_dEpsilon_D,ETA_Shape_P,ETA_Shape_Y,ETA_BetaP_s,ETA_Beta_ns_d,ETA_EtaP_F,ETA_Eta_B,ETA_EtaP_B,ETA_EtaP_B_RC,FMF,nHBAcc,IC1,IC2,IC3,SIC0,SIC1,SIC2,SIC3,MIC0,MIC1,Kier3,nAtomLC,nAtomP,nAtomLAC,MDEC-11,MDEO-11,MDEO-22,MDEN-22,MDEN-23,MDEN-33,MLFER_A,MLFER_BH,MLFER_S,R_TpiPCTPC,PetitjeanNumber,n6Ring,nHeteroRing,n6HeteroRing,RotBFrac,nRotBt,RotBtFrac,LipinskiFailures,GGI10,JGI1,JGI2,JGI3,JGI4,JGI5,JGI6,JGI7,JGI8,JGI9,JGI10,VE1_D,VR3_D,TopoPSA,XLogP,PubchemFP2,PubchemFP3,PubchemFP12,PubchemFP13,PubchemFP16,PubchemFP19,PubchemFP20,PubchemFP24,PubchemFP38,PubchemFP179,PubchemFP185,PubchemFP192,PubchemFP193,PubchemFP199,PubchemFP206,PubchemFP207,PubchemFP258,PubchemFP259,PubchemFP260,PubchemFP261,PubchemFP299,PubchemFP338,PubchemFP340,PubchemFP357,PubchemFP375,PubchemFP377,PubchemFP378,PubchemFP379,PubchemFP391,PubchemFP392,PubchemFP401,PubchemFP431,PubchemFP476,PubchemFP485,PubchemFP494,PubchemFP510,PubchemFP519,PubchemFP526,PubchemFP528,PubchemFP542,PubchemFP548,PubchemFP566,PubchemFP575,PubchemFP580,PubchemFP593,PubchemFP644,PubchemFP652,PubchemFP679,PubchemFP681,PubchemFP688,PubchemFP691,PubchemFP699,PubchemFP713,PubchemFP714,PubchemFP716,PubchemFP717,PubchemFP718,PubchemFP721,PubchemFP729,PubchemFP737,PubchemFP738,PubchemFP739,PubchemFP742,PubchemFP750,PubchemFP752,PubchemFP755,PubchemFP759
0,CHEMBL1170766,N#Cc1ccc(Nc2ncnc3c(=N)n(NC(=O)c4ccncc4)cnc23)cc1,17.20,active,7.764472,0,-1.4971,2.241308,25.6387,52.810309,22,9,1,0,0.0,0.0,0.0,0.0,4775.959573,243.972222,113.713323,116.429357,7.994673,7.949217,7.962848,1.673864,164.277002,152.122722,160.081051,164.443902,161.084591,4.228482,0.830810,-0.433699,-0.033613,0.256874,-0.460764,0.275811,0.168680,-0.334968,0.228552,26.333202,-136.475554,-97.376348,-249.195233,102.570119,-283.805452,-146.715397,28.025075,31.133953,-211.315642,-526.380834,-549.930239,606.001226,-170.476674,-2

In [3]:
df.set_index('molecule_chembl_id', inplace=True)

In [4]:
#df = df.drop(columns=['canonical_smiles','standard_value', 'bioactivity_class'])
#df.head()

In [5]:
features = ['BCUTw-1l', 'GATS7c', 'AATS8s', 'GATS4c', 'GATS8i', 'GATS6p']

X = df[features]
y = df['pIC50']

# Scale the features
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=df.index)


In [6]:
# # Scaffols chosen to make sure they are in the train data
# chosen_scaffolds = ['CHEMBL1087717', 'CHEMBL226983', 'CHEMBL3780092', 'CHEMBL3780593', 'CHEMBL196076',
#                     'CHEMBL197852', 'CHEMBL226069', 'CHEMBL3330937', 'CHEMBL64', 'CHEMBL1171167',
#                     'CHEMBL197852', 'CHEMBL227499', 'CHEMBL1172698', 'CHEMBL390333']  
# ## Get indices of chosen scaffolds from the DataFrame index
# chosen_indices = df.index[df.index.isin(chosen_scaffolds)].tolist()

# # Selecting chosen scaffolds from X_scaled and y
# X_chosen = X_scaled.loc[chosen_indices]
# y_chosen = y.loc[chosen_indices]

# # Select remaining data excluding chosen indices
# remaining_indices = [i for i in X_scaled.index if i not in chosen_indices]
# X_remaining = X_scaled.loc[remaining_indices]
# y_remaining = y.loc[remaining_indices]

# train_test_split on the remaining data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Combine chosen scaffolds with the training set from remaining data
# X_train = pd.concat([X_chosen, X_train_remaining], ignore_index=False)
# y_train = pd.concat([y_chosen, y_train_remaining], ignore_index=False)

print("X_train:", X_train.shape)
print("y_train", y_train.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape) 

X_train: (48, 6)
y_train (48,)
X_test: (12, 6)
y_test: (12,)


In [15]:
from sklearn.ensemble import AdaBoostRegressor

working_model2 = AdaBoostRegressor(
    base_estimator= DecisionTreeRegressor(max_depth=3),
    n_estimators=50,
    learning_rate=1,
    loss='square',
    random_state=42 )

working_model2.fit(X_train, y_train) 

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=3),
                  learning_rate=1, loss='square', random_state=42)

In [8]:
#X_train

In [16]:
import pickle

# Save model
with open('working_model2.pkl', 'wb') as f:
    pickle.dump(working_model2, f)  

# Save scaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

- Two compounds of interest are prediceted using their SMILES. 
- Compound 1 and 2.

In [29]:
%%time

import joblib  

new_smiles = ['C1=CN=CC=C1C(NN=CNC2=CC(=CC(=C2)OC)OC)=O',
'C1=CN=CC=C1C(NN=NC2=CC=CC=C2)=O']

new_smiles_df = pd.DataFrame({'canonical_smiles': new_smiles})
new_smiles_df.to_csv('compounds.smi', index=None, header=None)

# Generate descriptors (using PadelDescriptor or any other descriptor generator)
padeldescriptor(mol_dir='compounds.smi', d_file='descriptors_1.csv', d_2d=True,
 d_3d=False, fingerprints=True, retainorder=True)

# Load the pre-calculated descriptors for new SMILES
descriptors = pd.read_csv('descriptors_1.csv')

# Check the shape of new descriptors
print(f"Shape of new descriptors: {descriptors.shape}")

# Preprocessing
if 'Name' in descriptors.columns:
    descriptors = descriptors.drop('Name', axis=1)
    print("The 'Name' column has been dropped.")

# Select the features used
features = ['BCUTw-1l', 'GATS7c', 'AATS8s', 'GATS4c', 'GATS8i', 'GATS6p']

new_descriptors_selected = descriptors[features]

# Check for NaN or infinite values before scaling
if new_descriptors_selected.isnull().values.any():
    new_descriptors_selected = new_descriptors_selected.dropna()
    print("Warning: NaN values detected in descriptors and have been deleted")

# Load your trained scaler
scaler_ada = joblib.load('scaler.pkl')

# Scale the new descriptors using the loaded scaler
new_descriptors_scaled = scaler_ada.transform(new_descriptors_selected)

# Load your trained model
model_ada = joblib.load('working_model2.pkl')

# Make predictions using the scaled descriptors
predictions_pIC50 = model_ada.predict(new_descriptors_scaled)

# Convert predictions from pIC50 to nM
predictions_nM = 10 ** (9 - predictions_pIC50)

# Classify as active or not active based on the threshold of 1400 nM
activity_status = ["active" if pred < 1400 else "not active" for pred in predictions_nM]

# Display predictions and activity status
for pred, status in zip(predictions_pIC50, activity_status):
    print(f"Predicted activity (nM): {pred:.2f}, Status: {status}")


# for pred, status in zip(predictions_pIC50, activity_status):
#     print(f"Predicted activity (nM): {pred:.2f}, Status: {status}")

Shape of new descriptors: (2, 2326)
The 'Name' column has been dropped.
Predicted activity (nM): 8.96, Status: active
Predicted activity (nM): 8.96, Status: active
Wall time: 5.68 s
